In [2]:
import re
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import roc_auc_score
from TurkishStemmer import TurkishStemmer
import string

In [3]:
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
genesis_ic = wn.ic(genesis, False, 0.0)

[nltk_data] Downloading package genesis to /home/abs/nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package wordnet to /home/abs/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/abs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/abs/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Model Implementation

In [4]:
class KNN_NLC_Classifer():
    def __init__(self, k=3, distance_type = 'cosine'):
        self.k = k
        self.distance_type = distance_type

    # This function is used for training
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    # This function runs the K(1) nearest neighbour algorithm and
    # returns the label with closest match. 
    def predict(self, x_test):
        self.x_test = x_test
        y_predict = []

        for i in range(len(x_test)):
            max_sim = 0
            max_index = 0
            for j in range(self.x_train.shape[0]):
                temp = self.document_similarity(x_test[i], self.x_train[j])
                if temp > max_sim:
                    max_sim = temp
                    max_index = j
            y_predict.append(self.y_train[max_index])
        return y_predict

    def convert_tag(self, tag):
        """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
        tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
        try:
            return tag_dict[tag[0]]
        except KeyError:
            return None


    def doc_to_synsets(self, doc):
        """
            Returns a list of synsets in document.
            Tokenizes and tags the words in the document doc.
            Then finds the first synset for each word/tag combination.
        If a synset is not found for that combination it is skipped.

        Args:
            doc: string to be converted

        Returns:
            list of synsets
        """
        tokens = word_tokenize(doc+' ')
        
        l = []
        tags = nltk.pos_tag([tokens[0] + ' ']) if len(tokens) == 1 else nltk.pos_tag(tokens)
        
        for token, tag in zip(tokens, tags):
            syntag = self.convert_tag(tag[1])
            syns = wn.synsets(token, syntag)
            if (len(syns) > 0):
                l.append(syns[0])
        return l  

    def similarity_score(self, s1, s2, distance_type = 'cosine'):
          """
          Calculate the normalized similarity score of s1 onto s2
          For each synset in s1, finds the synset in s2 with the largest similarity value.
          Sum of all of the largest similarity values and normalize this value by dividing it by the
          number of largest similarity values found.

          Args:
              s1, s2: list of synsets from doc_to_synsets

          Returns:
              normalized similarity score of s1 onto s2
          """
          s1_largest_scores = []

          for i, s1_synset in enumerate(s1, 0):
              max_score = 0
              for s2_synset in s2:
                  if distance_type == 'path':
                      score = s1_synset.path_similarity(s2_synset, simulate_root = False)
                  elif distance_type == "cosine":
                      score = s1_synset.cosine_similarity(s2_synset)
                  else:
                      score = s1_synset.wup_similarity(s2_synset)                  
                  if score != None:
                      if score > max_score:
                          max_score = score
              
              if max_score != 0:
                  s1_largest_scores.append(max_score)
          
          mean_score = np.mean(s1_largest_scores)
                 
          return mean_score  

    def document_similarity(self,doc1, doc2):
          """Finds the symmetrical similarity between doc1 and doc2"""

          synsets1 = self.doc_to_synsets(doc1)
          synsets2 = self.doc_to_synsets(doc2)
          
          return (self.similarity_score(synsets1, synsets2) + self.similarity_score(synsets2, synsets1)) / 2

In [5]:
FILENAME = "data/grup1-2-3 veriseti.csv"          

dataset = pd.read_csv(FILENAME, header = None)

dataset.rename(columns = {0:'text', 1:'answer'}, inplace = True)

dataset['output'] = np.array(dataset["answer"])
for i in range(dataset.shape[0]):
    label = dataset.loc[i,"output"]
    if type(label) == float:
        dataset.drop([i], axis=0)
        continue
    label = re.sub("Y", "", label)
    dataset.loc[i,"output"] = label
    
Num_Words = dataset.shape[0]

print(dataset[:50])
print("\nDosyadaki Cümle Sayısı ", dataset.shape)

                                                 text answer output
0   Joe Saul ile Ed arkalarında yağan kar ile eşik...     Y6      6
1   Sert sürekli bir yel sanki mavi göklerden yağı...     Y6      6
2   Yaklaşan fırtınada yüzünü acımasızca ısıran su...     Y6      6
3   Kuzey Okyanusunun balina gemilerinden çıkmış İ...     Y1      1
4   Üzerine cehennem ateşleri yağdırır gibi bağırı...     Y6      6
5   Kare şeklinde dizilmiş kulübelerin tam merkezi...     Y1      1
6   Bu düşünceyle kendinden geçerek ayak parmaklar...     Y6      6
7   O zamanlar ateşte ufak et ve yağ parçalarını p...     Y1      1
8   Sonra Kaptan Morgan geminin ön tarafında menzi...     Y6      6
9   Halkın sevgisini satın almaya kalkıştığı halde...     Y6      6
10  Bir çiçek kopararak havaya fırlattım taç yapra...     Y6      6
11  Tanrı ölümsüz yağı verdi bize birimiz diğerimi...     Y2      2
12            Bildiği bütün lânetleri yağdıracak size     Y6      6
13  Tıpkı “Gel şu balıkyağını iç” der gibi söylü

In [5]:
dataset

,text,answer,output
0,Joe Saul ile Ed arkalarında yağan kar ile eşik...,Y6,6
1,Sert sürekli bir yel sanki mavi göklerden yağı...,Y6,6
2,Yaklaşan fırtınada yüzünü acımasızca ısıran su...,Y6,6
3,Kuzey Okyanusunun balina gemilerinden çıkmış İ...,Y1,1
4,Üzerine cehennem ateşleri yağdırır gibi bağırı...,Y6,6
...,...,...,...
5205,İnsan öldüğünde ilk önce yağları toprağa karışır,Y3,3
5206,Ölüden toprağa karışan yağ bir çok insana besi...,Y3,3
5207,Yağlı bir iş bulmuş torpili içeriden ayarlamıştı,Y5,5
5208,Üstümüzdeki gök olmasa göktaşları başımıza yağ...,Y6,6


# Preprocessing

In [6]:
s = set(stopwords.words('turkish'))
ps = nltk.wordnet.WordNetLemmatizer()
stemmer = TurkishStemmer()
for i in range(dataset.shape[0]):
    review = dataset.loc[i,'text']
    for punc in string.punctuation:
        review.replace(punc, "")
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in s]
    #review = [stemmer.stem(word) for word in review if not word in s]
    review = ' '.join(review)
    dataset.loc[i, 'text'] = review
X_train = dataset['text']
y_train = dataset['output']
print("Below is the sample of training text after removing the stop words")
print(dataset['text'][:10])

Below is the sample of training text after removing the stop words
0    joe saul ed arkalarında yağan kar eşikte durmu...
1    sert sürekli bir yel mavi göklerden yağıyor vâ...
2    yaklaşan fırtınada yüzünü acımasızca ısıran su...
3    kuzey okyanusunun balina gemilerinden çıkmış i...
4    üzerine cehennem ateşleri yağdırır bağırır bağ...
5    kare şeklinde dizilmiş kulübelerin tam merkezi...
6    düşünceyle kendinden geçerek ayak parmaklarını...
7    zamanlar ateşte ufak et yağ parçalarını pişire...
8    sonra kaptan morgan geminin ön tarafında menzi...
9    halkın sevgisini satın almaya kalkıştığı halde...
Name: text, dtype: object


In [10]:
dataset

,text,answer,output
0,joe saul ed arkalarında yağan kar eşikte durmu...,Y6,6
1,sert sürekli bir yel mavi göklerden yağıyor vâ...,Y6,6
2,yaklaşan fırtınada yüzünü acımasızca ısıran su...,Y6,6
3,kuzey okyanusunun balina gemilerinden çıkmış i...,Y1,1
4,üzerine cehennem ateşleri yağdırır bağırır bağ...,Y6,6
...,...,...,...
5205,i̇nsan öldüğünde ilk önce yağları toprağa karışır,Y3,3
5206,ölüden toprağa karışan yağ bir insana besin ol...,Y3,3
5207,yağlı bir iş bulmuş torpili içeriden ayarlamıştı,Y5,5
5208,üstümüzdeki gök olmasa göktaşları başımıza yağ...,Y6,6


In [11]:
del dataset["answer"]

In [12]:
dataset

,text,output
0,joe saul ed arkalarında yağan kar eşikte durmu...,6
1,sert sürekli bir yel mavi göklerden yağıyor vâ...,6
2,yaklaşan fırtınada yüzünü acımasızca ısıran su...,6
3,kuzey okyanusunun balina gemilerinden çıkmış i...,1
4,üzerine cehennem ateşleri yağdırır bağırır bağ...,6
...,...,...
5205,i̇nsan öldüğünde ilk önce yağları toprağa karışır,3
5206,ölüden toprağa karışan yağ bir insana besin ol...,3
5207,yağlı bir iş bulmuş torpili içeriden ayarlamıştı,5
5208,üstümüzdeki gök olmasa göktaşları başımıza yağ...,6


# Build

In [7]:
classifier = KNN_NLC_Classifer(k=3, distance_type='cosine')
classifier.fit(X_train, y_train)

# Test

In [8]:
final_test_list = ['yağ içti', "arabayı yağladı","makine yağı yere döküldü", "gazyağı lambasını yaktı", "yağ ter içinde kaldı", "kılıcını yağladı","Joe Saul ile Ed arkalarında yağan kar ile eşikte durmuş üstlerini başlarını silkiyorlardı"]
                 
test_corpus = []
for i in range(len(final_test_list)):
    review = final_test_list[i]
    for punc in string.punctuation:
        review.replace(punc, "")
    review = review.lower()
    review = review.split()
    #review = [stemmer.stem(word) for word in review if not word in s]
    review = [stemmer.stem(word) for word in review if not word in s]
    review = ' '.join(review)
    test_corpus.append(review)

In [9]:
y_pred_final = classifier.predict(test_corpus)

output_df = pd.DataFrame(data = {'text': final_test_list, 'code': y_pred_final})
#output_df['answer'] = np.where(output_df['code']==1, 'Doğal yağ','Mineral yağ', "Vücut yağı", "Kokulu yağ", "Mecaz", "Fiil")
print(output_df)

/home/abs/projects/data_mining/kara_classify/dm_venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/abs/projects/data_mining/kara_classify/dm_venv/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


                                                text code
0                                           yağ içti    6
1                                    arabayı yağladı    6
2                           makine yağı yere döküldü    2
3                            gazyağı lambasını yaktı    6
4                               yağ ter içinde kaldı    6
5                                   kılıcını yağladı    6
6  Joe Saul ile Ed arkalarında yağan kar ile eşik...    6


In [10]:
y_pred_final

['6', '6', '2', '6', '6', '6', '6']